<a href="https://colab.research.google.com/github/Guiandreis/pneumonia_detection/blob/main/time_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORT KAGGLE DATASET AND EXTRACT

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

{}

In [ ]:
! mkdir ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle/’: File exists


In [ ]:
! cp kaggle.json ~/.kaggle/ 


In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

ref                                                                title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
victorsoeiro/netflix-tv-shows-and-movies                           Netflix TV Shows and Movies                           2MB  2022-05-15 00:01:23          16031        464  1.0              
ruchi798/data-science-job-salaries                                 Data Science Job Salaries                             7KB  2022-06-15 08:59:12           5208        181  1.0              
zusmani/petrolgas-prices-worldwide                                 Petrol/Gas Prices Worldwide                          10KB  2022-06-24 01:25:33           2047         97  1.0              
imoore/age-dataset                           

In [ ]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

chest-xray-pneumonia.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile

In [ ]:
path_to_zip_file = '/content/chest-xray-pneumonia.zip'
directory_to_extract_to = '/content/'
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(directory_to_extract_to)

# IMPORT PACKAGES

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
import zipfile
import cv2
import random
import os

In [ ]:
test = glob.glob('/content/chest_xray/train/*/*')
len(test)

5216

In [ ]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from collections import defaultdict
import copy

import shutil
from urllib.request import urlretrieve

import albumentations as A
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch  
from torchmetrics import F1Score
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim
from torch.utils.data import  DataLoader,Dataset,random_split
import torchvision.models as models
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix


cudnn.benchmark = True

# CONFIGURING DATA SET

In [ ]:
train_path = glob.glob('/content/chest_xray/train/*/*')
test_path = glob.glob('/content/chest_xray/test/*/*')
val_path = glob.glob('/content/chest_xray/val/*/*')
total_images_path = train_path + test_path + val_path

len(train_path),len(test_path),len(val_path),len(total_images_path)

In [ ]:
train,val,test = random_split(total_images_path,[len(train_path) ,len(val_path),len(test_path)],generator = torch.Generator().manual_seed(42))

len(train),len(test),len(val),len(total_images_path)

(5216, 624, 16, 5856)

In [ ]:
'''
def display_image_grid(images_filepaths, predicted_labels=(), cols=5,samples = 2):
    rows = len(images_filepaths)+1 // cols
    figure, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(12, 6))
    k = 0
    for i, image_filepath in enumerate(images_filepaths):
        image = cv2.imread(image_filepath)
        print(image.shape)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        true_label = os.path.normpath(image_filepath).split(os.sep)[-2]
        predicted_label = predicted_labels[i] if predicted_labels else true_label
        color = "green" if true_label == predicted_label else "red"
        ax.ravel()[i].imshow(image,cmap='gray')
        ax.ravel()[i].set_title(predicted_label, color=color)
        ax.ravel()[i].set_axis_off()
        k+= 1
        if k >10:
          break
    plt.tight_layout()
    plt.show()

display_image_grid(test_path[:10])
'''

'\ndef display_image_grid(images_filepaths, predicted_labels=(), cols=5,samples = 2):\n    rows = len(images_filepaths)+1 // cols\n    figure, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(12, 6))\n    k = 0\n    for i, image_filepath in enumerate(images_filepaths):\n        image = cv2.imread(image_filepath)\n        print(image.shape)\n        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)\n        true_label = os.path.normpath(image_filepath).split(os.sep)[-2]\n        predicted_label = predicted_labels[i] if predicted_labels else true_label\n        color = "green" if true_label == predicted_label else "red"\n        ax.ravel()[i].imshow(image,cmap=\'gray\')\n        ax.ravel()[i].set_title(predicted_label, color=color)\n        ax.ravel()[i].set_axis_off()\n        k+= 1\n        if k >10:\n          break\n    plt.tight_layout()\n    plt.show()\n\ndisplay_image_grid(test_path[:10])\n'

In [ ]:
class normalvspneumonia(Dataset):
    def __init__(self, images_filepaths, transform=None):
        self.images_filepaths = images_filepaths
        self.transform = transform
        

    def __len__(self):
        return len(self.images_filepaths)

    def __getitem__(self, idx):
        #size_expected = 512
        image_filepath = self.images_filepaths[idx]
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        #image = self.pad_image_to_expected_size(image,size_expected,'constant')
        if image_filepath.split('/')[-2] == "PNEUMONIA":
            label = 1.0
        else:
            label = 0.0
        
        if self.transform is not None:
            image = self.transform(image=image)["image"]

        image = np.transpose(image,(2,0,1)).astype(np.float32)
        image = torch.tensor(image,dtype = torch.float)
        return image, label

In [ ]:
IMG_SIZE = 224
train_transform = A.Compose(
    [
        #A.SmallestMaxSize(max_size=160),
        A.Resize(IMG_SIZE,IMG_SIZE),
        #A.HorizontalFlip(p = 0.5),
        #A.VerticalFlip( p =0.5),
        #A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        #A.RandomCrop(height=128, width=128),
        A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
        #A.RandomBrightnessContrast(p=0.5),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)), ##normalização da imagenet
     
        #A.pytorch.ToTensor(),
     
    ]
)

val_transform = A.Compose(
    [
        #.SmallestMaxSize(max_size=160),
        
        A.Resize(IMG_SIZE,IMG_SIZE),
        #A.CenterCrop(height=128, width=128),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        #A.pytorch.ToTensor(),
    ]
)


In [ ]:
val_dataset = normalvspneumonia(images_filepaths=val, transform=val_transform)

train_dataset = normalvspneumonia(images_filepaths=train, transform=train_transform)

test_dataset = normalvspneumonia(images_filepaths=test, transform=val_transform)


In [ ]:
### just in case of augumentation
'''
def visualize_augmentations(dataset, idx=0, samples=10, cols=5):
    dataset = copy.deepcopy(dataset)
    dataset.transform = A.Compose([t for t in dataset.transform if not isinstance(t, (A.Normalize))])
    rows = samples // cols
    figure, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(12, 6))
    for i in range(samples):
        image, _ = dataset[idx]
        ax.ravel()[i].imshow(image[0,:,:],cmap='gray')
        ax.ravel()[i].set_axis_off()
    plt.tight_layout()
    plt.show()

random.seed(42)
visualize_augmentations(train_dataset)
'''

"\ndef visualize_augmentations(dataset, idx=0, samples=10, cols=5):\n    dataset = copy.deepcopy(dataset)\n    dataset.transform = A.Compose([t for t in dataset.transform if not isinstance(t, (A.Normalize))])\n    rows = samples // cols\n    figure, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(12, 6))\n    for i in range(samples):\n        image, _ = dataset[idx]\n        ax.ravel()[i].imshow(image[0,:,:],cmap='gray')\n        ax.ravel()[i].set_axis_off()\n    plt.tight_layout()\n    plt.show()\n\nrandom.seed(42)\nvisualize_augmentations(train_dataset)\n"

In [ ]:
image_datasets=  {'train': (train_dataset),
                 'validation': (val_dataset)
                 }

In [ ]:
batch_size =  64

dataloaders = {
    'train': DataLoader(train_dataset, batch_size=batch_size, shuffle=True),
    'validation':DataLoader(val_dataset, batch_size=batch_size, shuffle=True),
}
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Loading model

In [ ]:
############## teste
class resnet50model(nn.Module):

  def __init__(self):
    super(resnet50model, self).__init__()
    self.feature_extractor = models.resnet50(pretrained = True)
    self.feature_extractor = torch.nn.Sequential(*(list(self.feature_extractor.children())[:-2]))

    self.classifier = nn.Sequential(
        nn.AdaptiveAvgPool2d(output_size=(1, 1)),
        nn.Flatten(),

        nn.Linear(2048, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, 2)
        )
    self.gradient = None
  def activations_hook(self,grad):
    self.gradient = grad

  def forward(self,images):

    x = self.feature_extractor(images) ### feature maps / ACTIVATION MAPS
    h = x.register_hook(self.activations_hook)
    x = self.classifier(x)  ## top layers
    return x

  def get_activation_gradients(self): ##a1, a2, a3...ak
    return self.gradient

  def get_activation(self,x):   ### A1,A2,A3 .. ak
    return self.feature_extractor(x)







In [ ]:
model_classification1 = resnet50model()
PATH_SAVED_MODEL_1= '/content/gdrive/MyDrive/projects/model_pneumonia_resnet50.pt'

model_classification1.load_state_dict(torch.load(PATH_SAVED_MODEL_1,map_location = device))
model_classification1.to(device)
model_classification1.eval()

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


resnet50model(
  (feature_extractor): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          

# TIME analysis

In [ ]:
import time

In [ ]:
test_dataset[1]

(tensor([[[-2.1179, -2.1179, -2.1008,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],
 
         [[-2.0357, -2.0357, -2.0182,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],
 
         [[-1.8044, -1.8044, -1.7870,  ..., -1.8044, -1.8044, -1.8044],
          [-1.8044, -1.8044,

In [ ]:
test_dataloader_single_image = DataLoader(test_dataset[0], batch_size=2)
#test_dataset = normalvspneumonia(images_filepaths=test, transform=val_transform)


In [ ]:
image_test.shape

torch.Size([1, 3, 224, 224])

In [ ]:
#time for 1 image

%%time

image_test = cv2.imread(test[0]).astype(np.float32)
image_test = cv2.resize(image_test,(224,224))
image_test = torch.from_numpy(image_test).unsqueeze(0)
image_test = np.transpose(image_test,(0,3,1,2)).to(device)
image_test

model_classification1.eval()
predicted = model_classification1(image_test)
pred_probs = torch.softmax(predicted, dim=1).cpu().data.numpy()




CPU times: user 426 ms, sys: 7.31 ms, total: 433 ms
Wall time: 471 ms


In [ ]:
pred_probs

array([[0., 1.]], dtype=float32)

In [ ]:
#time for 1 batch

%%time

label_lists = []
predicted_lits = []
for inputs, labels in test_dataloader:
  inputs = inputs.to(device)
  print(inputs.dtype)
  labels = labels.type(torch.LongTensor)
  #print('labels',labels)
  labels = labels.to(device)
  #
  labels_cpu = labels.cpu().numpy()
  predicted = model_classification1(inputs)
  #
  _,preds = torch.max(predicted, 1)
  
  pred_probs = torch.softmax(predicted, dim=1).cpu().data.numpy()
  break




torch.float32


In [ ]:
#time for all testset

%%time

label_lists = []
predicted_lits = []
for inputs, labels in test_dataloader:
  inputs = inputs.to(device)
  labels = labels.type(torch.LongTensor)
  #print('labels',labels)
  labels = labels.to(device)
  #
  labels_cpu = labels.cpu().numpy()
  predicted = model_classification1(inputs)
  #
  _,preds = torch.max(predicted, 1)
  
  pred_probs = torch.softmax(predicted, dim=1).cpu().data.numpy()




CPU times: user 2min 10s, sys: 7.1 s, total: 2min 17s
Wall time: 2min 17s


### CONVERT MODEL TO ONNX

In [ ]:
!pip install onnxruntime


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.2 MB 13.9 MB/s 


In [ ]:
!pip install onnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.1 MB 10.6 MB/s 


In [ ]:
import onnxruntime
import onnx

In [ ]:
onnx.__version__

'1.12.0'

In [ ]:
'''
dummy_input = torch.randn(1,3,224,224,requires_grad = True).to(device)

torch.onnx.export(model_classification1,
                  dummy_input,
                  '/content/gdrive/MyDrive/projects/onnx_model_pneumonia_resnet50.onnx',
                  export_params = True,
                  opset_version = 10,
                  do_constant_folding = True,
                  input_names = ['input'],
                  output_names = ['output'],
                  dynamic_axes = {'input': {0 : 'batch_size'},
                                  'output': {0: 'batch_size'}}

                  )
'''

In [ ]:
ort_session  = onnxruntime.InferenceSession('/content/gdrive/MyDrive/projects/onnx_model_pneumonia_resnet50.onnx')

In [ ]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(image_test)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(predicted), ort_outs[0], rtol=1e-03, atol=1e-05)
print("Exported model has been tested with ONNXRuntime, and the result looks good!")


Exported model has been tested with ONNXRuntime, and the result looks good!


In [ ]:
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(image_test)}
ort_outs = ort_session.run(None, ort_inputs)
img_out_y = ort_outs[0]

In [ ]:
ort_outs

[array([[-431.673  ,  485.15836]], dtype=float32)]

In [ ]:
from PIL import Image


In [ ]:
img_out_y = Image.fromarray(np.uint8((img_out_y[0] * 255.0).clip(0, 255)[0]), mode='L')


IndexError: ignored